In [1]:
import os
import cv2
import argparse
import numpy as np
from skimage import io
from google.colab.patches import cv2_imshow
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Criando os Bboxes

In [ ]:
from PIL import Image
import os

list_ignored_labels = [
    'LOCAL', 'DATA EMISSÃO', 'ASSINATURA DO EMISSOR', 'OBSERVAÇÕES'
]

def checkClass(name):
    if name == 'NÚMERO':
        return 0
    elif name == 'CÓDIGO':
        return 1
    elif name == 'NÚMERO LATERAL':
        return 2
    elif name == 'LOCAL':
        return 3
    elif name == 'DATA EMISSÃO':
        return 4
    elif name == 'ASSINATURA DO EMISSOR':
        return 5
    elif name == 'OBSERVAÇÕES':
        return 6
    else:
        return -1

def read_coordinates_gt(filename):
    coordinates = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(',', 4)
            if len(parts) == 5:
                try:
                    x, y, width, height = map(int, parts[:4])
                    label = parts[4].strip()
                    coordinates.append((x, y, width, height, label))
                except ValueError:
                    print(f"Error processing line: {line.strip()}")
    return coordinates

def crop_images(image_path, coordinates, code, output_dir):
    image = Image.open(image_path)
    code_output_dir = os.path.join(output_dir, code)
    os.makedirs(code_output_dir, exist_ok=True)

    last_ignored_label = None
    label_counters = {}

    num_cut_direct = 0
    for coordinate in coordinates:
        x, y, w, h, label = coordinate
        label_id = checkClass(label)

        if num_cut_direct < 3:
            label_id = num_cut_direct
            num_cut_direct += 1

        elif label in list_ignored_labels:
            last_ignored_label = label
            continue

        elif label_id == -1:
            label_id = checkClass(last_ignored_label)
            if label_id == -1 or last_ignored_label is None:
                continue

        if label_id not in label_counters:
            label_counters[label_id] = 1
        else:
            label_counters[label_id] += 1

        current_counter = label_counters[label_id]
        file_name = f'{label_id}.{current_counter}.jpg'

        left = max(x - 5, 0)
        top = max(y - 2, 0)
        right = x + w + 2
        bottom = y + h + 2

        cropped_image = image.crop((left, top, right, bottom))
        if label_id == 2:
            cropped_image = cropped_image.rotate(270, expand=True)

        print(f"Save image: {file_name}")
        cropped_image.save(os.path.join(code_output_dir, file_name))

folder_path = '/content/drive/MyDrive/BD/BID Dataset/CNH_Verso'
new_gt = '/content/drive/MyDrive/news_gt_back'
output_dir = '/content/drive/MyDrive/cropped_images_back'
ignore_file_path = '/content/drive/MyDrive/dataset_back/ignore_files.txt'

with open(ignore_file_path, 'r', encoding='utf-8') as f:
    ignore_files = [line.strip() for line in f if line.strip()]

image_files = [
    f for f in os.listdir(folder_path)
    if f.endswith('.jpg') and '_gt_segmentation' not in f and f not in ignore_files
]

for image_file in image_files:
    print(f"Processing image: {image_file}")
    image_path = os.path.join(folder_path, image_file)
    image_code = os.path.splitext(image_file)[0].replace('_in', '')
    gt_file = os.path.join(new_gt, f"{image_code}_gt_ocr.txt")
    coordinates = read_coordinates_gt(gt_file)
    crop_images(image_path, coordinates, image_code, output_dir)


